In [1]:
from helper import DataPreprocessing
from ipywidgets import interact, IntSlider
from pyts.image import GramianAngularField
import matplotlib.pyplot as plt
import numpy as np

In [2]:
@interact(kind = ["train","full","test"], method = ["GASF","GADF"])
def choose_kind_data(kind,method):
    #load data
    data = DataPreprocessing(kind)
    
    #load method
    image_size = data.image_size
    if method == "GASF":
        GAF = GramianAngularField(method="summation",sample_range=None, image_size = image_size)
        mt = "Gramian Angular Summation Field"
    else:
        GAF = GramianAngularField(method="difference",sample_range=None, image_size = image_size)
        mt = "Gramian Angular Difference Field"

    if kind in ["train", "full"]:
        if kind == "train":
            bearing_number =  [11,12,21,22,31,32]
        else:
            bearing_number =  [13,14,15,16,17,23,24,25,26,27,33]
        @interact(bearing = bearing_number)
        def load_data_train(bearing):
            
            #load data
            N = data.n_samples(kind,bearing)
            eol = data.eol(bearing)
            time_stamp = np.linspace(0,eol,N)
            norm = data.norm(bearing)
            w_size = data.window_size_stft 
            hor_acc = np.clip(norm[0],-1,1)
            ver_acc = np.clip(norm[1],-1,1)
            sr = data.sr

            @interact(label = IntSlider(min=0,max=eol-90,step=10))
            def visualize_GAF (label):

                #scale data with label and step
                s_step = int(N - label * sr / 100  - w_size)
                time_stamp_w = time_stamp[s_step:s_step+w_size]
                time_stamp_w_real = np.linspace((eol-label)-90,(eol-label)+0.1,w_size)
                hor_acc_w = hor_acc[s_step:s_step+w_size]
                ver_acc_w = ver_acc[s_step:s_step+w_size]
                t_step = time_stamp_w_real.min()

                #visualize
                fig, ((ax1, ax2, ax3, ax4),(ax5,ax6, ax7,ax8)) = plt.subplots(nrows = 2, ncols = 4,figsize = (25,10))
                fig.suptitle("{} data, bearing {}, EOL {}s, {} samples\nMethod {}".format(kind,bearing,eol,N,mt))

                #axes 1:
                ax1.plot(time_stamp,hor_acc)
                ax1.plot(time_stamp_w,hor_acc_w)
                ax1.legend(["horizontal","window"],loc="upper left")
                ax1.set_ylabel("Normalized value")
                ax1.set_xlabel("Timestamp")
                ax1.set_title("Horizontal Acceleromete, time step {}".format(int(t_step)))

                #axes 2:
                ax2.plot(time_stamp_w_real,hor_acc_w,"darkorange")
                ax2.legend(["horizontal window"],loc = "upper left")
                ax2.set_ylabel("Normalized value")
                ax2.set_xlabel("Timestamp window")
                ax2.set_title("Horizontal Acceleromete Windowm, label {}".format(label))
                ax2.set_xticks([time_stamp_w_real.min(),time_stamp_w_real.max()])
                ax2.set_xticklabels([time_stamp_w_real.min(),time_stamp_w_real.max()])

                #axes 3:
                gaf_hor = GAF.fit_transform(hor_acc_w.reshape(1,-1))
                image_gaf_hor = ax3.imshow(gaf_hor.transpose(1,2,0))
                image_gaf_hor.set_clim(vmin=-1,vmax=1)
                plt.colorbar(image_gaf_hor,ax=ax3)
                ax3.set_title("{}, size {}".format(method,gaf_hor.shape))

                #axes 4:
                image_gaf_hor = ax4.imshow(gaf_hor.transpose(1,2,0))
                plt.colorbar(image_gaf_hor,ax=ax4)
                ax4.set_title("{}, real values".format(method,gaf_hor.shape))

                #axes 5:
                ax5.plot(time_stamp,ver_acc)
                ax5.plot(time_stamp_w,ver_acc_w)
                ax5.legend(["vertikal","window"],loc="upper left")
                ax5.set_ylabel("Normalized value")
                ax5.set_xlabel("Timestamp")
                ax5.set_title("Vertikal Acceleromete,time step {}".format(int(t_step)))
                
                #axes 6:
                ax6.plot(time_stamp_w_real,ver_acc_w,"darkorange")
                ax6.legend(["vertikal window"],loc = "upper left")
                ax6.set_ylabel("Normalized value")
                ax6.set_xlabel("Timestamp window")
                ax6.set_title("Horizontal Acceleromete Window label {}".format(label))
                ax6.set_xticks([time_stamp_w_real.min(),time_stamp_w_real.max()])
                ax6.set_xticklabels([time_stamp_w_real.min(),time_stamp_w_real.max()])

                #axes 7:
                gaf_ver = GAF.fit_transform(ver_acc_w.reshape(1,-1))
                image_gaf_ver = ax7.imshow(gaf_ver.transpose(1,2,0))
                image_gaf_ver.set_clim(vmin=-1,vmax=1)
                plt.colorbar(image_gaf_ver,ax=ax7)
                ax7.set_title("{}, size {}".format(method,gaf_hor.shape))

                #axes 8:
                image_gaf_ver = ax8.imshow(gaf_ver.transpose(1,2,0))
                plt.colorbar(image_gaf_ver,ax=ax8)
                ax8.set_title("{}, real values".format(method,gaf_hor.shape))

                print("Done")
                
    elif kind == "test":
        @interact(bearing = [13,14,15,16,17,23,24,25,26,27,33])
        def load_data_test(bearing):
            
            #load data
            N = data.n_samples(kind,bearing)
            last_cycle = data.last_cycle(bearing)
            eol = data.eol(bearing)
            rul = data.rul(bearing)
            time_stamp = np.linspace(0,last_cycle,N)
            norm = data.norm(bearing)
            w_size = data.window_size_gaf 
            hor_acc = norm[0]
            ver_acc = norm[1]
            sr = data.sr

            @interact(label = IntSlider(min=rul,max=rul+last_cycle-90,step=10))
            def visualize_GAF (label):

                #scale data with label and step
                s_step = int(N - (label-rul) * sr / 100  - w_size)
                time_stamp_w = time_stamp[s_step:s_step+w_size]
                time_stamp_w_real = np.linspace((eol-label)-90,(eol-label)+0.1,w_size)
                hor_acc_w = hor_acc[s_step:s_step+w_size]
                ver_acc_w = ver_acc[s_step:s_step+w_size]
                t_step = time_stamp_w_real.min()

                #visualize
                fig, ((ax1, ax2, ax3, ax4),(ax5,ax6, ax7,ax8)) = plt.subplots(nrows = 2, ncols = 4,figsize = (25,10))
                fig.suptitle("{} data, bearing {}, EOL {} s, RUL {} s, {} samples\nMethod {}".format(kind,bearing,eol,rul,N,mt))

                #axes 1:
                ax1.plot(time_stamp,hor_acc)
                ax1.plot(time_stamp_w,hor_acc_w)
                ax1.legend(["horizontal","window"],loc="upper left")
                ax1.set_ylabel("Normalized value")
                ax1.set_xlabel("Timestamp")
                ax1.set_title("Horizontal Acceleromete, step {}".format(int(t_step)))

                #axes 2:
                ax2.plot(time_stamp_w_real,hor_acc_w,c="darkorange")
                ax2.legend(["horizontal window"],loc = "upper left")
                ax2.set_ylabel("Normalized value")
                ax2.set_xlabel("Timestamp window")
                ax2.set_title("Horizontal Acceleromete Window, label {}".format(label))
                ax2.set_xticks([time_stamp_w_real.min(),time_stamp_w_real.max()])
                ax2.set_xticklabels([time_stamp_w_real.min(),time_stamp_w_real.max()])

                #axes 3:
                gaf_hor = GAF.fit_transform(hor_acc_w.reshape(1,-1))
                image_gaf_hor = ax3.imshow(gaf_hor.transpose(1,2,0))
                image_gaf_hor.set_clim(vmin=-1,vmax=1)
                plt.colorbar(image_gaf_hor,ax=ax3)
                ax3.set_title("{}, size {}".format(method,gaf_hor.shape))

                #axes 4:
                image_gaf_hor = ax4.imshow(gaf_hor.transpose(1,2,0))
                plt.colorbar(image_gaf_hor,ax=ax4)
                ax4.set_title("{}, real values".format(method,gaf_hor.shape))

                #axes 5:
                ax5.plot(time_stamp,ver_acc)
                ax5.plot(time_stamp_w,ver_acc_w)
                ax5.legend(["vertikal","window"],loc="upper left")
                ax5.set_ylabel("Normalized value")
                ax5.set_xlabel("Timestamp")
                ax5.set_title("Vertikal Acceleromete, step {}".format(int(t_step)))
                
                #axes 6:
                ax6.plot(time_stamp_w_real,ver_acc_w,c="darkorange")
                ax6.legend(["vertikal window"],loc = "upper left")
                ax6.set_ylabel("Normalized value")
                ax6.set_xlabel("Timestamp")
                ax6.set_title("Horizontal Acceleromete Window, label {}".format(label))
                ax6.set_xticks([time_stamp_w_real.min(),time_stamp_w_real.max()])
                ax6.set_xticklabels([time_stamp_w_real.min(),time_stamp_w_real.max()])
                
                #axes 7:
                gaf_ver = GAF.fit_transform(ver_acc_w.reshape(1,-1))
                image_gaf_ver = ax7.imshow(gaf_ver.transpose(1,2,0))
                image_gaf_ver.set_clim(vmin=-1,vmax=1)
                plt.colorbar(image_gaf_ver,ax=ax7)
                ax7.set_title("{}, size {}".format(method,gaf_hor.shape))

                #axes 8:
                image_gaf_ver = ax8.imshow(gaf_ver.transpose(1,2,0))
                plt.colorbar(image_gaf_ver,ax=ax8)
                ax8.set_title("{}, real values".format(method,gaf_hor.shape))

                print("Done")
                

interactive(children=(Dropdown(description='kind', options=('train', 'full', 'test'), value='train'), Dropdown…